In [0]:
%tensorflow_version 2.x


In [2]:

################################################################################
# Purpose: This code is for the implementation of the VAQ-E, that is given the #
# question and answer, it generates answer from the image and also gives       #
# textual explaination  In this part, I imported important liberaries, set TPU #
# instructions, mounted the drive, downloaded image folder and read the file   #
# names                                                                        #
#                                                                              #
# Disclaimer: No part of the code is copied from anywhere, I just have         #
# followed Tensorflow tutorials wherever I was stuck.                          # 
#                                                                              #
#                                                                              #
# Author: Akanksha Malhotra                                                    #
# Date: 20 April 2020                                                          #
#                                                                              #
################################################################################


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
import cv2
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.92.83.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.83.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
with open("/content/drive/My Drive/vqa/VQA-E-ComplementaryImages_train.json", "r") as read_file:
    training_data = json.load(read_file)

In [5]:
name_of_zip = 'train2014.zip'
if not os.path.exists(os.path.abspath('.') + '/' + name_of_zip):
  image_zip = tf.keras.utils.get_file(name_of_zip,
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip)+'/train2014/'
else:

  PATH = os.path.abspath('.')+'/train2014/'

13510574080/13510573713 [==============================] - 264s 0us/step


In [0]:
x = os.listdir("/content/train2014")

In [7]:

################################################################################
# Purpose: Reads the training data and gets full path for the image, answers,  # 
# questions, complementary image path, and explanation. There were some images #
# names in VQA-E dataset that were not available in train14 folder, image data #
# folder downloaded from MSCOCO.                                               #
#                                                                              #
# Author: Akanksha Malhotra                                                    #
# Date: 21 April 2020                                                          #
#                                                                              #
################################################################################

from tqdm import tqdm
questions  = []
answers = []
images = []
mcq = []
question_type = []
answer_type = []
explanation = []
complementary_pairs = []
from collections import defaultdict
image_path = []
all_answers = []
for i in tqdm(training_data):
    full_coco_image_path = "/content/train2014/" + 'COCO_train2014_' + '%012d.jpg' % (i["img_id"])
    c = 'COCO_train2014_' + '%012d.jpg' % (i["img_id"])
    if c not in x:
      continue
    d = defaultdict(int)
    for j in i["answers"]:
      d[j] += 1
    f=0
    for a in d.items():
      if a[1]>=8:
        v = a[0]
        f=1
    if f==0:
      continue
    else:
      all_answers.append(v)  
    questions.append(i['question'])
    images.append(i["img_id"])
    complementary_pairs.append(i["comp_img_id"])
    answers.append(max(d.items(), key=lambda x: x[1])[0])
    answer_type.append(i["answer_type"] )
    question_type.append(i["question_type"])
    mcq.append(i["multiple_choice_answer"])
    explanation.append(i["explanation"][0])
    image_path.append(full_coco_image_path)


100%|██████████| 160656/160656 [01:50<00:00, 1460.16it/s]


In [0]:
all_answers = set(all_answers)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
################################################################################
# Purpose: Shuffling the dataset to bring randomness in the data               # 
# Author: Akanksha Malhotra                                                    #
#                                                                              #
################################################################################
train_answers, train_questions, img_name_vector,explaination, all_complementary_pairs = shuffle(answers,questions,
                                          image_path,explanation, complementary_pairs,
                                          random_state=1)

# selecting the first 20000 examples from the shuffled set
num_examples = 1000
train_answers = train_answers[:num_examples]
train_questions = train_questions[:num_examples]
img_name_vector = img_name_vector[:num_examples]
all_complementary_pairs = all_complementary_pairs[:num_examples]
explaination = explaination[:num_examples]

In [0]:
import codecs
def load_glove_model(glove_file):
    """
    Given the path to the glove vector, it loads the glove embeddings
    Author: Akanksha Malhotra
    :param glove_file: embeddings_path: path of glove file.
    :return: glove model
    """

    f = codecs.open(glove_file + ".txt", 'r', encoding='utf-8')
    model = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        model[word] = embedding
    return model

In [11]:
def calc_max_length(tensor):
  ''' 
  Given the tensor, it returns maximum length of the sequence within the tensor
  Author: Akanksha Malhotra
  :param: tensor
  :return: max length of the sequence
  '''
  return max(len(t) for t in tensor)

################################################################################
# Purpose: For the questions in the dataset, it does preprocessing of the text.#
# The vocabulary is top 5000 words. Padding is added to generated sequence.    #
# The sequence of words is converted to sequence of numbers using word2index   #
# dictionary. 1 is added to vocab length to keep track of padding that we have #
# added.                                                                       #
#                                                                              #
# Author: Akanksha Malhotra                                                    #
################################################################################
top_k1 = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k1,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_questions)

print(tokenizer.word_index)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
ques_vocab = top_k1+1

train_question_seqs = tokenizer.texts_to_sequences(train_questions)

question_vector = tf.keras.preprocessing.sequence.pad_sequences(train_question_seqs, padding='post')

max_length = calc_max_length(train_question_seqs)
print(max_length)

#new edit
max_q = max_length

{'<unk>': 1, 'the': 2, 'is': 3, 'what': 4, 'are': 5, 'this': 6, 'in': 7, 'on': 8, 'a': 9, 'many': 10, 'how': 11, 'color': 12, 'of': 13, 'there': 14, 'man': 15, 'picture': 16, 'people': 17, 'animal': 18, 'wearing': 19, 'these': 20, 'does': 21, 'room': 22, 'playing': 23, 'animals': 24, 'person': 25, 'doing': 26, 'bus': 27, 'kind': 28, 'plate': 29, 'sport': 30, 'or': 31, 'holding': 32, 'it': 33, 'photo': 34, 'cat': 35, 'dog': 36, 'to': 37, 'they': 38, 'have': 39, 'woman': 40, 'type': 41, 'boy': 42, 'that': 43, 'sitting': 44, 'shown': 45, 'plane': 46, 'pizza': 47, 'ground': 48, 'water': 49, 'girl': 50, 'train': 51, 'at': 52, 'sign': 53, 'an': 54, 'white': 55, 'being': 56, 'horses': 57, 'can': 58, 'he': 59, 'game': 60, 'computer': 61, 'any': 62, 'seen': 63, 'elephants': 64, 'standing': 65, 'with': 66, 'shirt': 67, 'table': 68, 'be': 69, 'sky': 70, 'toilet': 71, 'hand': 72, 'bird': 73, 'zebras': 74, 'do': 75, "man's": 76, 'cake': 77, 'you': 78, 'her': 79, 'hat': 80, 'she': 81, 'riding': 82, 

In [12]:

# considering all answers to be part of ans vocab
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


###############################################################################
# Purpose: creates the label for each answer and creates answer vocab
#  
# Author: Akanksha Malhotra
###############################################################################

# define example
data = train_answers
values = array(data)
print(values[:10])

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

#new edit
ans_vocab = {l: i for i, l in enumerate(label_encoder.classes_)}

answer_vector = integer_encoded
print(answer_vector[:10])
len_ans_vocab = len(ans_vocab)

['cat' '2' 'yes' 'yes' 'sandwich' 'white' 'brown' 'frisbee' 'yes' 'no']
[ 43   2 191 191 138 185  34  73 191 115]


In [13]:

################################################################################
# Purpose: For the explanations in the dataset, it does preprocessing of the   #
# text. The vocabulary is top 5000 words. Padding is added to generated        #
# sequence.                                                                    #
# The sequence of words is converted to sequence of numbers using word2index   #
# dictionary. 1 is added to vocab length to keep track of padding that we have #
# added.                                                                       #
#                                                                              #
# Author: Akanksha Malhotra                                                    #
################################################################################

top_k = 5000
tokenizer1 = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer1.fit_on_texts(explaination)
train_explaination_seqs = tokenizer1.texts_to_sequences(explaination)

tokenizer1.word_index['<pad>'] = 0
tokenizer1.index_word[0] = '<pad>'


train_explaination_seqs = tokenizer1.texts_to_sequences(explaination)


explaination_vector = tf.keras.preprocessing.sequence.pad_sequences(train_explaination_seqs, padding='post')
print(tokenizer1.word_index)
explaination_vocab = top_k+1
max_length1 = calc_max_length(train_explaination_seqs)
print(max_length1)

#new edit
max_q1 = max_length1

{'<unk>': 1, 'a': 2, 'the': 3, 'is': 4, 'on': 5, 'in': 6, 'of': 7, 'with': 8, 'and': 9, 'are': 10, 'two': 11, 'man': 12, 'that': 13, 'to': 14, 'white': 15, 'sitting': 16, 'an': 17, 'standing': 18, 'at': 19, 'people': 20, 'there': 21, 'it': 22, 'next': 23, 'black': 24, 'woman': 25, 'holding': 26, 'plate': 27, 'table': 28, 'his': 29, 'street': 30, 'red': 31, 'blue': 32, 'down': 33, 'field': 34, 'this': 35, 'kitchen': 36, 'bathroom': 37, 'front': 38, 'bus': 39, 'playing': 40, 'some': 41, 'young': 42, 'has': 43, 'one': 44, 'large': 45, 'person': 46, 'cat': 47, 'dog': 48, 'top': 49, 'pizza': 50, 'grass': 51, 'green': 52, 'three': 53, 'baseball': 54, 'boy': 55, 'bed': 56, 'by': 57, 'men': 58, 'riding': 59, 'near': 60, 'wearing': 61, 'train': 62, 'frisbee': 63, 'up': 64, 'for': 65, 'while': 66, 'water': 67, 'tennis': 68, 'skateboard': 69, 'parked': 70, 'sign': 71, 'small': 72, 'snow': 73, 'air': 74, 'room': 75, 'other': 76, 'laptop': 77, 'looking': 78, 'walking': 79, 'group': 80, 'flying': 81

In [0]:
################################################################################
# Purpose: Split the data into validation and training with split percentage   #
# of 10%                                                                       #
# Author: Akanksha Malhotra                                                    #
################################################################################
img_name_train, img_name_val, question_train, question_val,answer_train, answer_val, explaination_train, explaination_val, all_comp, all_comp_val  = train_test_split(img_name_vector,
                                                                    question_vector,
                                                                    answer_vector,explaination_vector, all_complementary_pairs,
                                                                    test_size=0.1,
                                                                    random_state=0)

In [0]:
def map_func(img_name, ques,ans,exp):
  """
    Uses cv2 to read the image and resize it
    Creates one hot encoder for answers and explanation vector
    Author: Akanksha Malhotra
    :param:  image_name: path to the image, ques: ques_vector, ans: answer, 
             exp: explanation.
    :return: image_tensor, question, answer and explanation
    """
  img_tensor = np.array(cv2.resize(cv2.imread(img_name, cv2.IMREAD_UNCHANGED),(224,224)), dtype=np.float32)
  ans =  tf.one_hot(ans,depth=len(all_answers))
  exp =  [tf.one_hot(j,depth=explaination_vocab) for j in exp]
  return(img_tensor,ques, ans, exp)




In [16]:
def pretrained_embeddings(file_path, embedding_dim, vocab_Size, word2idx):
  '''
   1. load in pre-trained word vectors
   2. Creates embedding matrix for the given vocab using word2idx
   Author: Akanksha Malhotra
    :param:  file_path, embedding_dim, vocab_Size, word2idx
    :return: embedding matrix
  '''     
  glove_vectors = load_glove_model(file_path)
  print('Found %s word vectors.' % len(glove_vectors))
  print('Filling pre-trained embeddings...')
  num_words = vocab_Size
  # initialization by zeros
  embedding_matrix = np.zeros((num_words, embedding_dim))
  for word, i in word2idx.items():
    if i < vocab_Size:
        embedding_vector = glove_vectors.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all zeros.
          embedding_matrix[i] = embedding_vector
  return embedding_matrix
      
ques_embedding_matrix = pretrained_embeddings("/content/drive/My Drive/glove.6B.300d", 300, ques_vocab, tokenizer.word_index)



Found 400001 word vectors.
Filling pre-trained embeddings...


In [17]:
########################################################
# 1. Read Image                                        #
# 2. Create One-hot vector for answer and explanation. #
#                                                      #
# Author: Akanksha Malhotra                            # 
########################################################
with strategy.scope():
    ans_val = []
    wierd_img = []
    image_tensor_val = []
    ques_val = []
    exp_val = []
    for i in tqdm(range(len(img_name_val))):
      l = map_func(img_name_val[i], question_val[i], answer_val[i],explaination_val[i])
      if len(l[0].shape) ==  3:
        ans_val.append(l[2])
        image_tensor_val.append(l[0])
        ques_val.append(l[1]) 
        exp_val.append(l[3])
      else:
        wierd_img.append(i)
    # wierd_img = []
    ans = []
    image_tensor = []
    ques = []
    exp = []

    for i in tqdm(range(len(img_name_train))):
      l = map_func(img_name_train[i], question_train[i], answer_train[i],explaination_train[i])
      if len(l[0].shape) ==  3:
        ans.append(l[2])
        image_tensor.append(l[0])
        ques.append(l[1]) 
        exp.append(l[3])
      else:
        wierd_img.append(i)

    

100%|██████████| 900/900 [00:18<00:00, 48.63it/s]


In [0]:
########################################################
# 1. The dataset for training is created.              #
# 2. The dataset for validation  is created.           #
#                                                      #
# Author: Akanksha Malhotra                            # 
########################################################
# dataset = tf.data.Dataset.from_tensor_slices(({"ques": ques, "img":  image_tensor}, {"ans":ans,"exp":exp}))
dataset = tf.data.Dataset.from_tensor_slices(({"ques": ques, "img":  image_tensor}, {"ans":ans,"exp":exp}))
BATCH_SIZE = 1 #2 #64
BUFFER_SIZE = 300 #1000
num_steps = len(img_name_train) // BATCH_SIZE
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


# val_dataset = tf.data.Dataset.from_tensor_slices(({"ques": ques_val, "img":  image_tensor_val}, {"ans":ans_val,"exp":exp_val}))
# val_dataset = val_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [0]:
################################################################################
#  1. As the model has two inputs, two seperate input channels are created for #
#     tensorflow functional model.                                             # 
#     a. input1: Denotes the question tensor vector                            #
#     b. input2: Denotes the image tensor vector                               #
#  2. The pretrained model for images is loaded, we use ResNet152              #
#  3. Layers are created for the model                                         #
#        Question Layer: Embedding Layer, GRU and a Dense Layer                #
#        Image Layer: ResNet152 base model and a Dense Layer                   #
#        Attention: Attention Layer, Dense Layer, Hadamard Product             #
#        Answer generation: Attention , softmax                                #
#        Explanation generation: attention, dense, embedding layer, gru, dense #
#        with units equal to gru units and another dense layer with units equal#
#        to explanation vocab                                                  #
#  4. Compiled Model, with two losses                                          #
#  5. Model Fit
#  Author: Akanksha Malhotra                                                   #    
################################################################################ 

input1 =  keras.Input(shape=(max_q), name="ques", batch_size = BATCH_SIZE)
input2 =  keras.Input(shape=(224, 224, 3), name="img", batch_size = BATCH_SIZE)

print("read inputs")

IMG_SHAPE = (224, 224, 3)
base_model = keras.applications.ResNet152(input_shape=IMG_SHAPE, include_top=False, weights='imagenet', pooling='avg')
base_model.trainable = False

print("Basel Model of ResNet152 loaded")

embedding_dim = 300
enc_units = 1024
ques_model = layers.Embedding(ques_vocab, embedding_dim, weights=[ques_embedding_matrix],  trainable=False, input_length=max_q)(input1)
ques_model = layers.GRU(enc_units,recurrent_initializer='glorot_uniform')(ques_model)
ques_model = layers.Dense(1024, activation='relu')(ques_model)
img_model = base_model(input2)
img_model = layers.Dense(units=1024, activation="relu")(img_model)
attention = layers.Attention()([img_model, img_model, ques_model])
attention = layers.Dense(1024, activation='relu')(attention)
attention_output = layers.Multiply()([ques_model,attention])
at = layers.Dense(max_length1, activation='relu')(attention_output)
predicted_output = layers.Dense(1024,"relu")(attention_output)
predicted_output = layers.Dense(len(all_answers),"softmax",name="ans")(predicted_output)
emb = tf.keras.layers.Embedding(max_length1, embedding_dim)(at)
predicted_explaination = layers.GRU(enc_units,recurrent_initializer='glorot_uniform', return_sequences=True)(emb)
predicted_explaination = layers.Dense(enc_units)(predicted_explaination)
predicted_explaination1 = layers.Dense(explaination_vocab, name="exp")(predicted_explaination)

print("Model Architecture Created")

model = tf.keras.Model(inputs=[input1,input2], outputs=[predicted_output,predicted_explaination1])
print(model.summary())


losses = {'ans':tf.losses.categorical_crossentropy, 'exp': tf.losses.categorical_crossentropy}
model.compile(loss=losses,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

print("Model Compiled")

model.fit(dataset, epochs=50, verbose=1)

read inputs
234700800/234698864 [==============================] - 1s 0us/step
Basel Model of ResNet152 loaded
Model Architecture Created
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ques (InputLayer)               [(1, 13)]            0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (1, 13, 300)         1500300     ques[0][0]                       
__________________________________________________________________________________________________
img (InputLayer)                [(1, 224, 224, 3)]   0                                            
__________________________________________________________________________________________________
gru (GRU)                       (1, 1024)            40

898/898 [==============================] - 229s 254ms/step - loss: 18.6856 - ans_loss: 4.8611 - exp_loss: 13.8245 - ans_accuracy: 0.2572 - exp_accuracy: 0.1817
Epoch 2/50
608/898 [===================>..........] - ETA: 1:12 - loss: 16.1035 - ans_loss: 3.6512 - exp_loss: 12.4524 - ans_accuracy: 0.2467 - exp_accuracy: 0.1126

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/VQA - Project/data/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/VQA - Project/data/model.h5")
print("Saved model to disk")